In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
# Open sqlite file chat.db and read data from table messages into dataframe
group_chat_query = """
select 
    datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate,
    m.text,
    m.handle_id,
    h.id as "Sender",
    m.service,
    m.is_from_me,
    cmj.message_id,
    cmj.chat_id,
    c.service_name,
    c.group_id,
    m.associated_message_type,
    m.associated_message_guid,
    m.attributedBody
from message m 
left join chat_message_join cmj on m.ROWID = cmj.message_id  
left join chat c on cmj.chat_id = c.ROWID
left join handle h on m.handle_id = h.ROWID 
where c.group_id = 'F10BB39C-461D-4E72-8D4C-8718998118CF'
and m.service = c.service_name
order by "date" ASC
"""
msg_df = pd.read_sql_query(group_chat_query, sqlite3.connect("chat.db"))


In [ ]:

msg_df.head()

In [ ]:
msg_df['attributedBody']

In [ ]:
# Deserialize the attributedBody (NSAttributedString) column
from Foundation import NSData, NSUnarchiver, NSAttributedString

def deserialize_attributed_string(blob_data):
    # Handle when blob_data is None
    if blob_data is None:
        return None
    
    ns_data = NSData.dataWithBytes_length_(blob_data, len(blob_data))
    if ns_data is None:
        raise ValueError("Unable to create NSData from blob")

    attributed_string = NSUnarchiver.unarchiveObjectWithData_(ns_data)
    if not isinstance(attributed_string, NSAttributedString):
        raise ValueError("Deserialized object is not an NSAttributedString")

    return attributed_string.string()

# Use the deserialize_attributed_string function to deserialize the attributedBody column
msg_df['text'] = msg_df['attributedBody'].apply(deserialize_attributed_string)


In [ ]:
# Show rows where text is null, but attributedBody is not null
# Ahow only the text and attributedBody columns
msg_df[msg_df['text'].isnull() & msg_df['attributedBody'].notnull()][['text', 'attributedBody']]

In [ ]:
# Replace null text with empty string
msg_df['text'].fillna('', inplace=True)

In [ ]:
# Replace NaN in Sender column with value 2547221897
msg_df['Sender'].fillna('+12547221897', inplace=True)

In [ ]:
# Map Sender column to names where +1254... is Clive, +1757... Yousef, and +1408... is Daniel
msg_df['Sender'] = msg_df['Sender'].map({'+12547221897': 'Clive', '+17576606447': 'Yousef', '+14088328217': 'Daniel'})

In [ ]:
# Count the number of messages send by each Sender, only count the text column
msg_df.groupby('Sender')['text'].count()

In [ ]:
# Get values in the 'text' column that start with a single word, a space then a phrase in double-quotes such as 'Liked "Hello World"'
msg_df[msg_df['text'].str.contains(r'^\w+\s“')]

In [ ]:
# Value rows where the text column starts with one of the following words: Liked, Loved, Laughed at, Emphasized, Questioned
msg_df[msg_df['text'].str.contains(r'^Liked|^Loved|^Laughed at|^Emphasized|^Questioned|^Disliked')]
# Now create a new column based on the word that was found. So if the text column starts with 'Liked', the new column should have the value 'Liked'
msg_df['Reaction'] = msg_df['text'].str.extract(r'^(Liked|Loved|Laughed at|Emphasized|Questioned|^Disliked)')
# Show me the rows with Reactions
msg_df[msg_df['Reaction'].notnull()]

In [ ]:
# Now count the number of times each reaction was used
msg_df.groupby('Reaction')['text'].count()

In [ ]:
# List all unique associated_message_type values
msg_df['associated_message_type'].unique()

In [ ]:
# Get the text, Reaction, and associated_message_type for each row where there is a reaction
msg_df[msg_df['Reaction'].notnull()][['text', 'Reaction', 'associated_message_type']]

In [ ]:
# Get unique reaction and associated_message_type combinations and their counts
msg_df[msg_df['Reaction'].notnull()].groupby(['Reaction', 'associated_message_type'])['text'].count()

In [ ]:
# OK so now we know that associated_message_type indicates the type of reaction. 
# It seems 2000 = Loved, 2001 = Liked, 2002 = Disliked, 2003 = Laughed at, 2004 = Emphasized, 2005 = Questioned
# There are some where there associated_message_type is 0, but I think that means its not a reaction message (regex captured too much)

In [ ]:
# Find the Sender who sends the most reactions
msg_df[msg_df['Reaction'].notnull()].groupby('Sender')['text'].count()

In [ ]:
# For each reaction type, find the Sender who sends the most of that reaction
reaction_names = {
    2000: 'Loved',
    2001: 'Liked',
    2002: 'Disliked',
    2003: 'Laughed at',
    2004: 'Emphasized',
    2005: 'Questioned'
}
for reaction_type, reaction_name in reaction_names.items():
    print(reaction_name)
    print(msg_df[(msg_df['Reaction'].notnull()) & (msg_df['associated_message_type'] == reaction_type)].groupby('Sender')['text'].count().sort_values(ascending=False))
    print()

In [ ]:
# Next we need to use the message_id and associated_message_guid to find the message that was reacted to
# We can use the message_id to find the message that was reacted to
# We can use the associated_message_guid to find the reaction message
# We can use the associated_message_type to find the type of reaction
# We can use the chat_id to find the group chat
# We can use the handle_id to find the Sender


Ok another case I just found is "React an image". It's always "an image". not super interesting but I'll come back to it.

Also found a case for "Removed a Reaction". I saw there are 300x types, that's probably what that means

In [ ]:
# Print datatype of each column
msg_df.dtypes